In [137]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
import numpy as np

In [4]:
def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [6]:
dataset = list(parse(os.path.join("data", "trainRecipes.json.gz")))

In [7]:
len(dataset)

200000

In [165]:
dataframe = pd.DataFrame(dataset)
dataframe

,name,minutes,contributor_id,submitted,steps,description,ingredients,recipe_id
0,sexy fried eggs for sunday brunch,10,14298494,2004-05-21,heat a ridged griddle pan\tlightly brush the t...,"this is from silvana franco's book ""family"" wh...","[plum tomato, ciabatta, olive oil, egg, garlic...",06432987
1,double delicious cookie bars,40,26865936,2007-08-27,preheat oven to 350f\tin 13x9-inch baking pan ...,"from ""all time favorite recipes"". for fun, try...","[butter, graham cracker crumbs, sweetened cond...",98015212
2,gorgeous chocolate muffins,50,11206582,2008-05-28,"in a bowl cream butter , sugar , vanilla sugar...",this is a true treat for all chocolate lovers!...,"[butter, sugar, vanilla sugar, egg, milk, vine...",27596743
3,amy s quick kielbasa skillet dinner,25,53076165,2003-10-03,combine frozen potatoes and sliced kielbasa in...,i made this up one night when i needed a quick...,"[frozen southern style hash brown potatoes, ki...",74041916
4,old fashioned chocolate bread pudding with kah...,55,56214328,2008-10-08,preheat oven to 375f\tbegin by buttering 12 sm...,are you tired of serving the 'same old - same ...,"[butter, granulated sugar, whole milk, heavy c...",52053290
...,...,...,...,...,...,...,...,...
199995,green chile enchiladas,70,51649462,2002-12-19,"cook beef and garlic powder in skillet , stirr...",quick and easy ground beef dinner.,"[lean ground beef, garlic powder, colby-monter...",62771271
199996,leftover ham with pasta and broccoli casserole,37,97438710,2007-04-10,preheat oven to 425 degrees\tcook pasta accord...,great use for leftover easter ham.,"[spiral shaped pasta, cubed ham, frozen brocco...",70012752
199997,oatmeal cookies with raisins and cranberries,70,71977973,2008-02-10,heat oven to 350f\tbeat together butter and su...,i adapted this recipe for my husband who adore...,"[butter, brown sugar, granulated sugar, eggs, ...",45070289
199998,polenta pizza with brie and red grapes,195,85742694,2009-02-11,combine yeast with 1 / 4 cup warm water in med...,i positively love this recipe! this is for whe...,"[yeast, warm water, olive oil, kosher salt, al...",93801897


In [13]:
train = dataset[:150000]
valid = dataset[150000:175000]
test = dataset[175000:]

In [ ]:
### Question 1

In [46]:
#1a
def feat1a(d):
    dataframe = pd.DataFrame(train)
    dataframe['length of recipe'] = dataframe['steps'].apply(len)
    dataframe['ingredients count'] = dataframe['ingredients'].apply(len)
    print(dataframe['length of recipe'][0])
    print(dataframe['ingredients count'][0])
    regr = linear_model.LinearRegression()
    X = dataframe[['length of recipe', 'ingredients count']]
    Y = dataframe['minutes']
    regr.fit(X, Y)
    
    test_df = pd.DataFrame(d)
    test_df['length of recipe'] = test_df['steps'].apply(len)
    test_df['ingredients count'] = test_df['ingredients'].apply(len)
    x_pred = list(regr.predict(test_df[['length of recipe', 'ingredients count']]))
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

feat1a(test)

743
9


6169.549296366498

In [56]:
#1b
def feat1b(d):
    dataframe = pd.DataFrame(train)
    dataframe['submitted'] = dataframe['submitted'].apply(lambda x: pd.to_datetime(x))
    dataframe['month'] = dataframe['submitted'].apply(lambda x: x.month)
    dataframe['year'] = dataframe['submitted'].apply(lambda x: x.year)
    print(dataframe[['month','year']].iloc[0])
    ct = ColumnTransformer([('ohe', OneHotEncoder(drop = 'first'), ['month', 'year'])])
    pl = Pipeline([('ct', ct), ('lr', LinearRegression())])
    pl.fit(dataframe[['month','year']], dataframe['minutes'])
    
    test_df = pd.DataFrame(d)
    test_df['submitted'] = test_df['submitted'].apply(lambda x: pd.to_datetime(x))
    test_df['month'] = test_df['submitted'].apply(lambda x: x.month)
    test_df['year'] = test_df['submitted'].apply(lambda x: x.year)
    x_pred = pl.predict(test_df[['month','year']])
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

feat1b(test)

month       5
year     2004
Name: 0, dtype: int64


6396.83368942199

In [94]:
#1c
def feat1c(d):
    dataframe = pd.DataFrame(train)
    total_ingred = [ingred for sublist in dataframe['ingredients'] for ingred in sublist]
    pop_ingred = pd.DataFrame(list(pd.DataFrame(total_ingred, columns = ['most popular ingred']).value_counts().reset_index()['most popular ingred']), columns = ['most_pop'])[:50]
    binary = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binary[i] = dataframe['ingredients'].apply(lambda x: 1 if i in x else 0)
    print(binary.iloc[0])
    regr = linear_model.LinearRegression()
    X = binary 
    Y = dataframe['minutes']
    regr.fit(X,Y)
    
    test_df = pd.DataFrame(d)
    binarycopy = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binarycopy[i] = test_df['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    x_pred = list(regr.predict(binarycopy))
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

feat1c(test)


salt                         0
butter                       0
sugar                        0
onion                        0
water                        0
eggs                         0
olive oil                    1
flour                        0
milk                         0
garlic cloves                0
pepper                       0
brown sugar                  0
garlic                       0
all-purpose flour            0
baking powder                0
egg                          1
salt and pepper              1
parmesan cheese              0
lemon juice                  0
baking soda                  0
vegetable oil                0
vanilla                      0
black pepper                 0
cinnamon                     0
sour cream                   0
tomatoes                     0
garlic powder                0
vanilla extract              0
oil                          0
garlic clove                 1
honey                        0
cream cheese                 0
onions  

6000.948439855976

In [111]:
#Question 2
#for all features 
def ablation_for_all(d):
    dataframe = pd.DataFrame(train)
    dataframe['length of recipe'] = dataframe['steps'].apply(len)
    dataframe['ingredients count'] = dataframe['ingredients'].apply(len)
    
    dataframe['submitted'] = dataframe['submitted'].apply(lambda x: pd.to_datetime(x))
    dataframe['month'] = dataframe['submitted'].apply(lambda x: x.month)
    dataframe['year'] = dataframe['submitted'].apply(lambda x: x.year)
    
    total_ingred = [ingred for sublist in dataframe['ingredients'] for ingred in sublist]
    pop_ingred = pd.DataFrame(list(pd.DataFrame(total_ingred, columns = ['most popular ingred']).value_counts().reset_index()['most popular ingred']), columns = ['most_pop'])[:50]
    binary = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binary[i] = dataframe['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    features_df = dataframe[['length of recipe', 'ingredients count', 'month', 'year']]
    features_df = pd.concat([features_df, binary], axis = 1)
    
    ct = ColumnTransformer([('ohe', OneHotEncoder(drop = 'first'), ['month', 'year'])], remainder = "passthrough")
    pl = Pipeline([('ct', ct), ('lr', LinearRegression())])
    pl.fit(features_df, dataframe['minutes'])
    
    regr = linear_model.LinearRegression()
    X = features_df
    Y = dataframe['minutes']
    regr.fit(X,Y)
    
    test_df = pd.DataFrame(d)
    test_df['length of recipe'] = test_df['steps'].apply(len)
    test_df['ingredients count'] = test_df['ingredients'].apply(len)
    
    test_df['submitted'] = test_df['submitted'].apply(lambda x: pd.to_datetime(x))
    test_df['month'] = test_df['submitted'].apply(lambda x: x.month)
    test_df['year'] = test_df['submitted'].apply(lambda x: x.year)
    
    binarycopy = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binarycopy[i] = test_df['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    predictor = pd.concat([test_df[['length of recipe','ingredients count','month','year']], binarycopy], axis = 1)
    
    x_pred = list(pl.predict(predictor))
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

all_f = ablation_for_all(test)
all_f

5861.253905671388

In [112]:
#for no a)
def ablation_1a(d):
    dataframe = pd.DataFrame(train)
    
    dataframe['submitted'] = dataframe['submitted'].apply(lambda x: pd.to_datetime(x))
    dataframe['month'] = dataframe['submitted'].apply(lambda x: x.month)
    dataframe['year'] = dataframe['submitted'].apply(lambda x: x.year)
    
    total_ingred = [ingred for sublist in dataframe['ingredients'] for ingred in sublist]
    pop_ingred = pd.DataFrame(list(pd.DataFrame(total_ingred, columns = ['most popular ingred']).value_counts().reset_index()['most popular ingred']), columns = ['most_pop'])[:50]
    binary = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binary[i] = dataframe['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    features_df = dataframe[['month', 'year']]
    features_df = pd.concat([features_df, binary], axis = 1)
    
    ct = ColumnTransformer([('ohe', OneHotEncoder(drop = 'first'), ['month', 'year'])], remainder = "passthrough")
    pl = Pipeline([('ct', ct), ('lr', LinearRegression())])
    pl.fit(features_df, dataframe['minutes'])
    
    regr = linear_model.LinearRegression()
    X = features_df
    Y = dataframe['minutes']
    regr.fit(X,Y)
    
    test_df = pd.DataFrame(d)
    
    test_df['submitted'] = test_df['submitted'].apply(lambda x: pd.to_datetime(x))
    test_df['month'] = test_df['submitted'].apply(lambda x: x.month)
    test_df['year'] = test_df['submitted'].apply(lambda x: x.year)
    
    binarycopy = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binarycopy[i] = test_df['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    predictor = pd.concat([test_df[['month','year']], binarycopy], axis = 1)
    
    x_pred = list(pl.predict(predictor))
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

ab1a = ablation_1a(test)
ab1a

5992.663510100702

In [113]:
#difference between all features and 1b & 1c
all_f - ab1a

-131.40960442931464

In [115]:
#for no b)
def ablation_1b(d):
    dataframe = pd.DataFrame(train)
    dataframe['length of recipe'] = dataframe['steps'].apply(len)
    dataframe['ingredients count'] = dataframe['ingredients'].apply(len)
    
    total_ingred = [ingred for sublist in dataframe['ingredients'] for ingred in sublist]
    pop_ingred = pd.DataFrame(list(pd.DataFrame(total_ingred, columns = ['most popular ingred']).value_counts().reset_index()['most popular ingred']), columns = ['most_pop'])[:50]
    binary = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binary[i] = dataframe['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    features_df = dataframe[['length of recipe', 'ingredients count']]
    features_df = pd.concat([features_df, binary], axis = 1)
    
    regr = linear_model.LinearRegression()
    X = features_df
    Y = dataframe['minutes']
    regr.fit(X,Y)
    
    test_df = pd.DataFrame(d)
    test_df['length of recipe'] = test_df['steps'].apply(len)
    test_df['ingredients count'] = test_df['ingredients'].apply(len)
    
    binarycopy = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binarycopy[i] = test_df['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    predictor = pd.concat([test_df[['length of recipe','ingredients count']], binarycopy], axis = 1)
    
    x_pred = list(regr.predict(predictor))
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

ab1b = ablation_1b(test)
ab1b

5870.11506165606

In [116]:
#difference between all features and 1a & 1c
all_f - ab1b

-8.861155984672223

In [118]:
#for no c):
def ablation_1c(d):
    dataframe = pd.DataFrame(train)
    dataframe['length of recipe'] = dataframe['steps'].apply(len)
    dataframe['ingredients count'] = dataframe['ingredients'].apply(len)
    
    dataframe['submitted'] = dataframe['submitted'].apply(lambda x: pd.to_datetime(x))
    dataframe['month'] = dataframe['submitted'].apply(lambda x: x.month)
    dataframe['year'] = dataframe['submitted'].apply(lambda x: x.year)
    
    features_df = dataframe[['length of recipe', 'ingredients count', 'month', 'year']]
    
    ct = ColumnTransformer([('ohe', OneHotEncoder(drop = 'first'), ['month', 'year'])], remainder = "passthrough")
    pl = Pipeline([('ct', ct), ('lr', LinearRegression())])
    pl.fit(features_df, dataframe['minutes'])
    
    regr = linear_model.LinearRegression()
    X = features_df
    Y = dataframe['minutes']
    regr.fit(X,Y)
    
    test_df = pd.DataFrame(d)
    test_df['length of recipe'] = test_df['steps'].apply(len)
    test_df['ingredients count'] = test_df['ingredients'].apply(len)
    
    test_df['submitted'] = test_df['submitted'].apply(lambda x: pd.to_datetime(x))
    test_df['month'] = test_df['submitted'].apply(lambda x: x.month)
    test_df['year'] = test_df['submitted'].apply(lambda x: x.year)
    
    predictor = test_df[['length of recipe','ingredients count','month','year']]
    
    x_pred = list(pl.predict(predictor))
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

ab1c = ablation_1c(test)
ab1c

6157.753375895338

In [119]:
#difference between all features and 1a & 1b
all_f - ab1c

-296.4994702239501

In [123]:
#Q2 response 
#Of all the features, it seems like the 50D binary vector indicating the presence of the 50 most popular ingredients  
#is the most important for predicting minutes of a recipe since it has the biggest MSE when 1c feature is not included
#This is because when thinking logically, date and year does not really affect recipe minutes and while length
#of the steps does correlate with more instructions, it might not equivalate to more steps. The order of importance 
#of the features from least to most is [B, A, C]

In [126]:
#Question 3
def ablation_for_all(lamb, d):
    dataframe = pd.DataFrame(train)
    dataframe['length of recipe'] = dataframe['steps'].apply(len)
    dataframe['ingredients count'] = dataframe['ingredients'].apply(len)
    
    dataframe['submitted'] = dataframe['submitted'].apply(lambda x: pd.to_datetime(x))
    dataframe['month'] = dataframe['submitted'].apply(lambda x: x.month)
    dataframe['year'] = dataframe['submitted'].apply(lambda x: x.year)
    
    total_ingred = [ingred for sublist in dataframe['ingredients'] for ingred in sublist]
    pop_ingred = pd.DataFrame(list(pd.DataFrame(total_ingred, columns = ['most popular ingred']).value_counts().reset_index()['most popular ingred']), columns = ['most_pop'])[:50]
    binary = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binary[i] = dataframe['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    features_df = dataframe[['length of recipe', 'ingredients count', 'month', 'year']]
    features_df = pd.concat([features_df, binary], axis = 1)
    
    ct = ColumnTransformer([('ohe', OneHotEncoder(drop = 'first'), ['month', 'year'])], remainder = "passthrough")
    pl = Pipeline([('ct', ct), ('ridge', Ridge(alpha = lamb))])
    pl.fit(features_df, dataframe['minutes'])
    
    regr = linear_model.LinearRegression()
    X = features_df
    Y = dataframe['minutes']
    regr.fit(X,Y)
    
    test_df = pd.DataFrame(d)
    test_df['length of recipe'] = test_df['steps'].apply(len)
    test_df['ingredients count'] = test_df['ingredients'].apply(len)
    
    test_df['submitted'] = test_df['submitted'].apply(lambda x: pd.to_datetime(x))
    test_df['month'] = test_df['submitted'].apply(lambda x: x.month)
    test_df['year'] = test_df['submitted'].apply(lambda x: x.year)
    
    binarycopy = pd.DataFrame()
    for i in list(pop_ingred['most_pop']):
        binarycopy[i] = test_df['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    predictor = pd.concat([test_df[['length of recipe','ingredients count','month','year']], binarycopy], axis = 1)
    
    x_pred = list(pl.predict(predictor))
    mse = np.average((test_df['minutes'] - x_pred)**2)
    return mse

In [127]:
#q3 training 
training_lst = []
for lamb in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    training_lst.append(ablation_for_all(lamb, train))

training_lst
#based on the results, lambda = 0.001 has the best performance for the training set 

[5677.717036144234,
 5677.717036156725,
 5677.717037400427,
 5677.717156675642,
 5677.725411160895,
 5677.886706162655,
 5682.268247945253]

In [128]:
#q3 validation
validation_lst = []
for lamb in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    validation_lst.append(ablation_for_all(lamb, valid))

validation_lst
#based on the results, lambda = 10 has the best perfomrtance for the validation set 

[5680.854068641053,
 5680.854006645393,
 5680.853390511754,
 5680.847599737171,
 5680.817805211031,
 5681.206429355591,
 5689.903073676642]

In [129]:
#q3 testing using best lambda from validation 
ablation_for_all(10, test)

5861.502338522187

In [158]:
#question 5
def question5(d):
    df = pd.DataFrame(train)
    
    train_ingred = pd.DataFrame()
    total_ingred = [ingred for sublist in df['ingredients'] for ingred in sublist]
    pop_ingred = list(pd.DataFrame(list(pd.DataFrame(total_ingred, columns = ['most popular ingred']).value_counts().reset_index()['most popular ingred']), columns = ['most_pop'])[:50]['most_pop'])
    for i in pop_ingred:
        train_ingred[i] = df['ingredients'].apply(lambda x: 1 if i in x else 0)
    
    lr = LogisticRegression(class_weight = 'balanced')
    butter = train_ingred.pop('butter')
    lr.fit(train_ingred, butter)
    
    test_df = pd.DataFrame(d)
    test_ingred = pd.DataFrame()
    for i in pop_ingred:
        test_ingred[i] = test_df['ingredients'].apply(lambda x: 1 if i in x else 0)
        
    test_butter = test_ingred.pop('butter')
    x_pred = lr.predict(test_ingred)
    comparison = pd.DataFrame()
    comparison['actual'] = test_butter
    comparison['predicted'] = x_pred
    
    return comparison
    
y = question5(test)
y

,actual,predicted
0,0,0
1,0,1
2,0,1
3,0,0
4,0,0
...,...,...
24995,0,0
24996,1,1
24997,1,1
24998,0,0


In [159]:
#q5 computing balanced error rate
def BER (df):
    TP = df[(df['actual'] == 1) & (df['predicted'] == 1)].shape[0]
    TN = df[(df['actual'] == 0) & (df['predicted'] == 0)].shape[0]
    FP = df[(df['actual'] == 0) & (df['predicted'] == 1)].shape[0]
    FN = df[(df['actual'] == 1) & (df['predicted'] == 0)].shape[0]
    P = TP + FN
    N = TN + FP 
    return 0.5 * (FP/(FP+TN) + FN/(FN+TP))

BER(y)

0.28898437523315856

#Question 7
Accuracy will be important for this classifier because accuracy makes sure that we are looking at the correct recipes and allow us to identify the true positives and negatives. Precision will also be important for this classifier because it allows us to identify that out of the recipes returned which ones are relevant/important for our query. For example, when trying to find dietary restricted recipes precision is highlighted here when a bundle of recipes returned which ones are actually close to or actually following the dietary guidelines provided. 

In [166]:
#Question 8
ingsPerItem = defaultdict(set)
itemsPerIng = defaultdict(set)
for d in dataset:
    r = d['recipe_id']
    for i in d['ingredients']:
        ingsPerItem[r].add(i)
        itemsPerIng[i].add(r)

def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

def mostSimilar(i, N=5):
    similarities = []
    ings = ingsPerItem[i]
    for i2 in ingsPerItem:
        if i2 == i: continue
        sim = Jaccard(ings, ingsPerItem[i2])
        #sim = Pearson(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    sorting = pd.DataFrame(similarities[:N], columns = ['sim','recipe_id'])
    return sorting

mostSimilar('06432987', 5).sort_values(by = ['sim','recipe_id'], ascending =[False, True])
#this is soted by digits

,sim,recipe_id
0,0.416667,68523854
1,0.384615,12679596
3,0.363636,56301588
2,0.363636,79675099
4,0.357143,87359281


In [167]:
#question 9
def mostSimilaring(i, N=5):
    similarities = []
    recipe = itemsPerIng[i]
    for i2 in itemsPerIng:
        if i2 == i: continue
        sim = Jaccard(recipe, itemsPerIng[i2])
        #sim = Pearson(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    sorting = pd.DataFrame(similarities[:N], columns = ['sim','ingredients'])
    return sorting

mostSimilaring('butter', 5)

,sim,ingredients
0,0.223153,salt
1,0.205669,flour
2,0.191004,eggs
3,0.178824,sugar
4,0.170401,milk


In [219]:
#question 10
"""
Explanation: When inquiring the recipes based on the current ingredients, each ingredient will be checked if they are
in the top 50 most popular ingredients based on the dataset given. If the ingredient queried is not in the top 50 most
popular ingredients, then the function will perform a jaccard similarity to find the 5 most similar ingredients to the 
current ingredient inquired. Once all the ingredients have been queried, the ingredients which was resulted from 
jaccard will be check how frequent they appear in all the recipes, and half of the ingredients that are most frequent
within the list of jaccard inquired ingredients will be used to find which recipes use any of these recipes. Finally,
jaccard will be performed on the recipes that uses any of the jaccard-inquired-most-frequent ingredients from previous
section to find the most similar recipes. 
"""
def question10(j):
    total_ingred = [ingred for sublist in dataframe['ingredients'] for ingred in sublist]
    pop_ingred = pd.DataFrame(list(pd.DataFrame(total_ingred, columns = ['most popular ingred']).value_counts().reset_index()['most popular ingred']), columns = ['most_pop'])[:50]
    all_ingred = []
    for ingred in j:
        if ingred not in list(pop_ingred['most_pop']):
            all_ingred.append(ingred)
            jaccard_list = list(mostSimilaring(ingred, 5)['ingredients'])
            for each in jaccard_list:
                all_ingred.append(each)
    
    ingred_count = dict()
    for i in list(dataframe['ingredients']):
        for ing in all_ingred:
            if ing in i:
                ingred_count[ing] = ingred_count.get(ing, 0) +1
                
    ingred_count = sorted(ingred_count.items(), key=lambda x: x[1])[:len(ingred_count)//2]
    ingred_count = [each[0] for each in ingred_count]
    
    similar_recipes = []
    for i in ingred_count:
        recipes_df = pd.DataFrame()
        recipes_df['recipe_id'] = dataframe['recipe_id']
        recipes_df[i] = dataframe['ingredients'].apply(lambda x: 1 if i in x else 0)
        recipe = list(recipes_df[recipes_df[i] == 1]['recipe_id'])[0]
        similar_recipes.append(mostSimilar(recipe, 1)['recipe_id'][0])
    
    output = pd.DataFrame()
    output['recipe_id'] = similar_recipes
    output_names = []
    for reps in similar_recipes:
        output_names.append(list(dataframe[dataframe['recipe_id'] == str(reps)]['name'])[0])
    output['name'] = output_names
    return output
        
question10(['cinnamon', 'cherries', 'butterscotch', 'vodka'])
    

,recipe_id,name
0,26097636,honey i roasted the sunflower seeds
1,13366174,mom s delicious homemade bread
2,68592158,holiday garlic lemon green beans with bread cr...
3,92875126,garlic chicken n gravy
4,38200543,frangelico chocolate martini
5,84420998,simple pasta dough
6,96147320,rose sangria
7,38200543,frangelico chocolate martini
